In [1]:

import pandas as pd
import numpy as np

from prepare import extract_data 
from prepare import get_distance
from prepare import drop_columns
from prepare import drop_not_us
from prepare import address_split


In [2]:
df=extract_data()


Importing csv...


C:\Users\JARED~1.GOD\AppData\Local\Temp/ipykernel_7120/410605488.py:1: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  df=extract_data()
C:\Users\JARED~1.GOD\AppData\Local\Temp/ipykernel_7120/410605488.py:1: DtypeWarning: Columns (2,9) have mixed types.Specify dtype option on import or set low_memory=False.
  df=extract_data()


Importing json...
Data import complete.
Shape: (1000003, 22)


In [ ]:
df.head()

In [3]:
df.columns = df.columns.str.replace(' ','_')

In [4]:
# Convert Ticket number to string, Issue date to date, and display datatypes to confirm changes
df['Ticket_number'] = df['Ticket_number'].astype(str)
df['Issue_Date'] = pd.to_datetime(df['Issue_Date'])
# Drop LAT/LON == 9999
df = df[(df.Latitude != 99999) | (df.Longitude != 99999)]
# Define point A
lat2 = 6439997
lon2 = 1802686
# Create column calculating distance to above point
df['Distance_to_pointA']=round(get_distance(df['Latitude'], df['Longitude'], lat2, lon2), 2)

In [5]:
df.head()

,Ticket_number,Issue_Date,Issue_time,Meter_Id,Marked_Time,RP_State_Plate,Plate_Expiry_Date,VIN,Make,Body_Style,...,Agency,Violation_code,Violation_Description,Fine_amount,Latitude,Longitude,Agency_Description,Color_Description,Body_Style_Description,Distance_to_pointA
2,1104803000,2015-12-21,2055.0,NaN,NaN,CA,201503.0,NaN,NISS,PA,...,2.0,8939,WHITE CURB,58.0,6439997.9,1802686.4,NaN,NaN,NaN,0.98
3,1104820732,2015-12-26,1515.0,NaN,NaN,CA,NaN,NaN,ACUR,PA,...,2.0,0,17104h,NaN,6440041.1,1802686.2,NaN,NaN,NaN,44.10
6,1108321723,2015-12-21,1041.0,NaN,NaN,CA,201610.0,NaN,HOND,PA,...,1.0,8.06E+07,RED ZONE,93.0,6451860.6,1890192.1,NaN,NaN,NaN,88306.64
15,1109139006,2015-12-19,1550.0,NaN,NaN,CA,201608.0,NaN,HOND,NaN,...,1.0,225078A,HANDICAP/NO DP ID,363.0,6429130.8,1880027.3,NaN,NaN,NaN,78100.90
16,1109455266,2015-12-26,1815.0,NaN,NaN,NY,NaN,NaN,TOYO,PA,...,36.0,8603,PK IN PROH AREA,68.0,6467477.1,1868721.0,NaN,NaN,NaN,71524.66


In [6]:
df.drop_duplicates(inplace=True)


In [7]:
df = drop_columns(df, 0.7)

Columns over 0.7 null threshold to drop:  ['Marked_Time', 'VIN', 'Agency_Description', 'Color_Description', 'Body_Style_Description']
Original dimensions: (749978, 23)
Dimensions after drop: (749978, 18)
5 columns dropped.


In [8]:
df = drop_not_us(df)

Dropping license plates from outside the 50 US States
Original dimensions: (749978, 18)
Final dimensions: (748314, 18)
1664 rows dropped.


In [10]:
# Replace nulls with the mode for the violation code

df['Fine_amount'] = np.where(df.Violation_code == '0', 93,
                           np.where(df.Violation_code == '80.69.2', 78,
                                   np.where(df.Violation_code == '80.71.3', 68,
                                           np.where(df.Violation_code == '8069BS', 73,0))))

In [11]:
# split address
df = address_split(df)


In [12]:
# Make issue year column out of the datetime object issue date

df['Issue_year'] = df['Issue_Date'].dt.year

In [19]:
pd.set_option("display.max_columns", None)

In [20]:
df.head()

,Ticket_number,Issue_Date,Issue_time,Meter_Id,RP_State_Plate,Plate_Expiry_Date,Make,Body_Style,Color,Location,Route,Agency,Violation_code,Violation_Description,Fine_amount,Latitude,Longitude,Distance_to_pointA,address_0,address_1,address_2,address_3,address_4,address_5,address_6,address_7,Issue_year
2,1104803000,2015-12-21,2055.0,NaN,CA,201503.0,NISS,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE CURB,0,6439997.9,1802686.4,0.98,200,WORLD,WAY,None,None,None,None,None,2015
3,1104820732,2015-12-26,1515.0,NaN,CA,NaN,ACUR,PA,WH,100 WORLD WAY,2F11,2.0,0,17104h,93,6440041.1,1802686.2,44.10,100,WORLD,WAY,None,None,None,None,None,2015
6,1108321723,2015-12-21,1041.0,NaN,CA,201610.0,HOND,PA,BK,10635 FRIAR ST,15EL5,1.0,8.06E+07,RED ZONE,0,6451860.6,1890192.1,88306.64,10635,FRIAR,ST,None,None,None,None,None,2015
15,1109139006,2015-12-19,1550.0,NaN,CA,201608.0,HOND,NaN,GO,14006 RIVERSIDE DR,35MQ1,1.0,225078A,HANDICAP/NO DP ID,0,6429130.8,1880027.3,78100.90,14006,RIVERSIDE,DR,None,None,None,None,None,2015
16,1109455266,2015-12-26,1815.0,NaN,NY,NaN,TOYO,PA,WH,3200 CANYON DR,NaN,36.0,8603,PK IN PROH AREA,0,6467477.1,1868721.0,71524.66,3200,CANYON,DR,None,None,None,None,None,2015
